In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
import glob
import os

os.chdir('/Users/vivekkanpa/Documents/UCD/sudoku-imputation/SUDOKU_BENCHMARK/')
os.getcwd()

missingness = 'mnar'
dataset = 'eeg'

In [2]:
sudoku_dir = f"outputs/{missingness}/{dataset}"
sudoku_files = glob.glob(os.path.join(sudoku_dir, '*.txt'))

benchmark_dir = f"datasets/imputed_data/numerical/{dataset}/{missingness}/"
benchmark_files = glob.glob(os.path.join(benchmark_dir, '*.txt'))


In [3]:
sudoku_files

['outputs/mnar/eeg/execution_time_0.4.txt',
 'outputs/mnar/eeg/execution_time_0.5.txt',
 'outputs/mnar/eeg/execution_time_0.2.txt',
 'outputs/mnar/eeg/execution_time_0.3.txt',
 'outputs/mnar/eeg/execution_time_0.1.txt']

In [4]:
def read_text_files_into_list(lo_files):
    all_file_contents = dict()
    
    # Get all files in the directory
    for filename in lo_files:
        if filename.endswith(".txt"):  # Filter for text files
            try:
                with open(filename, 'r', encoding='utf-8') as file:
                    content = file.read()
                    if 'outputs' in filename:
                        all_file_contents[filename.split('_')[-1][0:-4]] = float(content.split(' ')[0])
                    else:
                        all_file_contents[filename.split('/')[-1].split('_')[2]] = float(content.split(' ')[0])
            except Exception as e:
                print(f"Error reading {filename}: {e}")
    return all_file_contents

all_runtimes = {'sudoku' : read_text_files_into_list(sudoku_files)}

In [5]:
# ['mean', 'matrix', 'knn', 'mice', 'softimpute', 'median']
for model in list(set([file.split('.')[1][2:].split('_')[0] for file in benchmark_files])):
    model_files = []
    for bm_file in benchmark_files:
        if model in bm_file:
            model_files.append(bm_file)
    all_runtimes[model] = read_text_files_into_list(model_files)

In [6]:
runtime_df = pd.DataFrame(all_runtimes).reset_index().sort_values(by='index').sort_values(by='index').rename(columns={'index':'MP'}).set_index('MP')
runtime_df

,sudoku,knn,mice,median,softimpute,matrix,mean,missforest
MP,,,,,,,,
0.1,9.2386,1.7821,19.4223,0.0150,1.0435,0.3069,0.0033,149.7296
0.2,11.3148,3.3904,16.9727,0.0215,1.4483,0.1878,0.0029,177.7716
0.3,17.6142,5.9343,17.6931,0.0145,1.0067,0.0539,0.0031,97.4065
0.4,59.9883,7.6388,17.2905,0.0141,1.1704,0.3663,0.0030,97.3102
0.5,73.0872,6.1929,17.8392,0.0138,1.7625,0.0717,0.0030,96.9065


In [3]:
runtime_df.to_csv(f'analysis/supp_tables/s2_mean_runtime_{dataset}_{missingness}.csv')

NameError: name 'runtime_df' is not defined

In [8]:
directory_path = f"analysis/results_tables/runtime/{dataset}"

try:
    os.makedirs(directory_path, exist_ok=True)
except OSError as e:
    print(f"Error creating directory '{directory_path}': {e}")

runtime_df.to_csv(f'{directory_path}/{missingness}_runtime_table.csv')